# Import Site-Package

In [ ]:
import random
import time,math
import numpy as np
import gymnasium as gym
import gymnasium.wrappers as gym_wrap
import matplotlib.pyplot as plt
import matplotlib.animation as animation #輸出動畫影片
from IPython import display
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torch.nn as nn
import collections

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# VAE

In [ ]:
input_size = 84 * 84
hidden_size = 800
latent_size = 64
class VAEEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, latent_size*2)

    def forward(self, x):
        x = self.fc2( torch.relu(self.fc1(x)) )
        mean, log_var = x.split(latent_size, dim=1)
        return mean, log_var
class VAEDecoder(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(latent_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.sigmoid( self.fc2( torch.relu(self.fc1(x)) ) )-0.5
        return x
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super().__init__()
        self.encoder = VAEEncoder(input_size, hidden_size, latent_size)
        self.decoder = VAEDecoder(latent_size, hidden_size, input_size)

    def forward(self, x):
        mean, log_var = self.encoder(x)
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std).to(device)
        z = mean + std * eps    #random sampling
        reconstruct = self.decoder(z)
        return reconstruct, mean, log_var
model = VAE(input_size, hidden_size, latent_size)
model.load_state_dict(torch.load(f'VAE_16000.pth',weights_only=True))

<All keys matched successfully>

# Initialize

### load model

In [ ]:
Load_File=0
Old_File=f"Model-{Load_File}.pt"
if Load_File>0:
  Log= np.load(f"Log-{Load_File}.npy", allow_pickle=True).item()
else:
  Log={"TrainReward":[],"TestReward":[],"Loss":[],"TestAction":[],"TestInfo":[]}

### env

In [ ]:
class ImageEnv(gym.Wrapper):
  def __init__(self,env,stack_frames=4,delay_op=50):
    super(ImageEnv, self).__init__(env)
    self.delay_op = delay_op
    self.stack_frames = stack_frames
  def reset(self):
    s, info = self.env.reset()
    for i in range(self.delay_op):
      s, r, terminated, truncated, info = self.env.step(0)
      s=torch.tensor((s[:84, 6:90]/255.0)-0.5).reshape((1,-1))

      with torch.no_grad():
        s, log_var = model.encoder(s.float())
      self.stacked_state = np.tile( s , (self.stack_frames,1) )  # [4, 84, 84]
    return self.stacked_state, info

  def step(self, action):
    reward = 0
    for _ in range(self.stack_frames):
      s, r, terminated, truncated, info = self.env.step(action)
      if r==-100:terminated=True
      s=torch.tensor((s[:84, 6:90]/255.0)-0.5).reshape((1,-1))
      with torch.no_grad():
        s, log_var = model.encoder(s.float())
      reward += r
      if terminated or truncated or r<-80:break
      self.stacked_state = np.concatenate((self.stacked_state[1:], s), axis=0)
    return self.stacked_state, reward, terminated, truncated, info

In [ ]:
env=gym.make('CarRacing-v3',render_mode="rgb_array",domain_randomize=False, continuous=False)
env = gym_wrap.GrayscaleObservation(env)
env = ImageEnv(env)

# Replay Buffer

In [ ]:
class ReplayBuffer:
  def __init__(self,max_size=int(1e5), num_steps=1):
    self.s = np.zeros((max_size,4,64), dtype=np.float32)
    self.a = np.zeros((max_size,), dtype=np.int64)
    self.r = np.zeros((max_size, 1), dtype=np.float32)
    self.s_ = np.zeros((max_size,4,64), dtype=np.float32)
    self.done = np.zeros((max_size, 1), dtype=np.float32)
    self.info = np.zeros((max_size, 7), dtype=np.float32)
    self.info_ = np.zeros((max_size, 7), dtype=np.float32)
    self.ptr = 0
    self.size = 0
    self.max_size = max_size
    self.num_steps = num_steps

  def append(self,s,a,r,s_,done,info,info_):
    self.s[self.ptr] = s
    self.a[self.ptr] = a
    self.r[self.ptr] = r
    self.s_[self.ptr] = s_
    self.done[self.ptr] = done
    self.info[self.ptr] = info
    self.info_[self.ptr] = info_
    self.ptr = (self.ptr + 1) % self.max_size
    self.size = min(self.size+1,self.max_size)

  def sample(self, batch_size):
    ind = np.random.randint(0, self.size, batch_size)
    return torch.FloatTensor(self.s[ind]),torch.LongTensor(self.a[ind]),torch.FloatTensor(self.r[ind]),torch.FloatTensor(self.s_[ind]),\
           torch.FloatTensor(self.done[ind]),torch.FloatTensor(self.info[ind]).to(device),torch.FloatTensor(self.info_[ind]).to(device)

# Sensor Fusion

In [ ]:
class SensorFusion(nn.Module):
    def __init__(self, output_dim=128):
        super(SensorFusion, self).__init__()
        self.fc_state = nn.Linear(256, output_dim)
        self.fc_info = nn.Linear(7, output_dim)
        self.fusion = nn.Linear(output_dim * 2, output_dim)

    def forward(self,state,info):
        state = state.view((-1,256)).to(device)
        info = info.view((-1,7)).to(device)

        x1 = self.fc_state(state)
        x2 = self.fc_info(info)
        fused = torch.cat((x1, x2), dim=1)
        fused = self.fusion(fused)
        return fused

# DQN

In [ ]:
class DQN(torch.nn.Module):
  def __init__(self,n_act,output_dim=128):
    super(DQN,self).__init__()
    self.SensorFusion = SensorFusion(output_dim=128)
    self.fc1 = torch.nn.Linear(output_dim, 256)
    self.fc2 = torch.nn.Linear(256, 64)

    self.value_fc=torch.nn.Linear(64,16)
    self.value=torch.nn.Linear(16,1)
    self.adv_fc=torch.nn.Linear(64,32)
    self.adv=torch.nn.Linear(32,n_act)

  def forward(self, x, info):
    fused_feature = self.SensorFusion.forward(x,info)
    x = fused_feature.view((-1,128))
    x = torch.relu(self.fc2(torch.relu(self.fc1(x))))
    value=self.value(F.relu(self.value_fc(x)))
    adv=self.adv(F.relu(self.adv_fc(x)))
    return value+(adv-adv.mean(dim=-1,keepdim=True))

# 搭建智能體Agent的類別

In [ ]:
class DQNAgent():
  def __init__(self,gamma=0.9,eps_low=0.1,lr=0.00025):
    self.env = env
    self.n_act=self.env.action_space.n
    self.PredictDQN= DQN(self.n_act)
    self.TargetDQN= DQN(self.n_act)
    if Load_File>0:
      self.PredictDQN.load_state_dict(torch.load(Old_File))
      self.TargetDQN.load_state_dict(torch.load(Old_File))
    self.PredictDQN.to(device)
    self.TargetDQN.to(device)
    self.LossFun=torch.nn.SmoothL1Loss()
    self.optimizer=torch.optim.Adam(self.PredictDQN.parameters(),lr=lr)
    self.gamma=gamma
    self.eps_low=eps_low
    self.rb=ReplayBuffer(max_size=10000, num_steps=1)

  def change_info_type(self,_):
      with torch.no_grad():
          teml = [_["true_speed"],_["four_ABS_sensors"][0],_["four_ABS_sensors"][1],_["four_ABS_sensors"][2],_["four_ABS_sensors"][3],_["steering_wheel_position"],_["gyroscop"]]
          for i in range(7): teml[i] = np.float64(teml[i])
          info = np.array(teml)
          return info

  def PredictA(self,s,info):
    with torch.no_grad():
      st=torch.FloatTensor(s).to(device)
      it=torch.FloatTensor(info).to(device)
      return torch.argmax(self.PredictDQN(st,it)).item()
  def SelectA(self,a):
    return self.env.action_space.sample() if np.random.random()<self.EPS else a

  def Train(self,N_EPISODES):
    for i in tqdm(range(Load_File,N_EPISODES)):
      self.EPS=self.eps_low+(1-self.eps_low)*math.exp(-i*12/(N_EPISODES))
      total_reward=0
      s,info=self.env.reset()
      info = self.change_info_type(info)
      while True:
        a=self.SelectA(self.PredictA(torch.FloatTensor(s),torch.FloatTensor(info)))
        s_,r,done,stop,info_=self.env.step(a)
        info_ = self.change_info_type(info_)
        self.rb.append(s,a,r,s_,done,info,info_)
        if self.rb.size > 200 and i%self.rb.num_steps==0:self.Learn()
        if i % 20==0:  self.TargetDQN.load_state_dict(self.PredictDQN.state_dict())
        s, info = s_, info_
        total_reward+=r
        if done or stop:break
      print(f"\n{total_reward}")
      Log["TrainReward"].append(total_reward)
      if i % 10 == 9:
        test_reward=self.Test()
        print(f"\n訓練次數{i+1}，總回報{test_reward}")
        Log["TestReward"].append(test_reward)
        torch.save(self.PredictDQN.state_dict(), f"Model-{i+1}.pt")
        np.save(f"Log-{i+1}.npy", Log)
  def Learn(self):
    self.optimizer.zero_grad()
    batch_s, batch_a, batch_r, batch_s_, batch_done, batch_info, batch_info_=self.rb.sample(32)
    predict_Q = (self.PredictDQN(batch_s.to(device),batch_info)*F.one_hot(batch_a.long().to(device),self.n_act)).sum(1,keepdims=True)
    with torch.no_grad():
      target_Q = batch_r.to(device)+(1-batch_done.to(device))*self.gamma*self.TargetDQN(batch_s_.to(device),batch_info_).max(1,keepdims=True)[0]
    loss = self.LossFun(predict_Q, target_Q)
    Log["Loss"].append(float(loss))
    loss.backward()
    self.optimizer.step()
  def Test(self,VIDEO=False):
    total_reward=0
    video=[]
    actionl,infol = [],[]
    s,info=self.env.reset()
    info = self.change_info_type(info)
    while True:
      video.append(self.env.render())
      a=self.PredictA(s,info)
      s,r,done,stop,info=self.env.step(a)
      info = self.change_info_type(info)
      actionl.append(a)
      infol.append(info)
      total_reward+=r
      if done or stop:
        Log["TestAction"].append(actionl)
        Log["TestInfo"].append(infol)
        break
    if VIDEO:
      patch = plt.imshow(video[0]) #產生展示圖形物件
      plt.axis('off') #關閉坐標軸
      def animate(i): #設定更換影格的函數
        patch.set_data(video[i])
        #plt.gcf()=>建新繪圖區 animate=>更換影格函數 frames=>影格數 interval=>影隔間距(毫秒)
      anim = animation.FuncAnimation(plt.gcf(),animate,frames=len(video),interval=200)
      anim.save('Car_Racing.mp4') #儲存為mp4擋
    return total_reward
  def Record(self):
    total_reward=0
    s,_=self.env.reset()
    while True:
      image=self.env.render()
      plt.imshow(image)
      #plt.imsave(f"/content/drive/MyDrive/recording/{str(int(time.time()))}.png", image)
      a=self.PredictA(s)
      s,r,done,stop,_=self.env.step(a)
      print(r)
      total_reward+=r
      plt.pause(0.1)
      #清除目前的顯示
      display.clear_output(wait=True)
      if done or stop:break
    print(total_reward)

In [ ]:
Agent=DQNAgent(gamma=0.95,eps_low=0.05,lr=0.00025)
Agent.Train(N_EPISODES=5000)

  0%|                                                                              | 1/5000 [00:16<23:09:47, 16.68s/it]


-34.62264150943473


  0%|                                                                              | 2/5000 [00:33<23:31:05, 16.94s/it]


-27.7681003584232


  0%|                                                                              | 3/5000 [00:40<17:05:40, 12.32s/it]

...


31.599999999999966


100%|██████████████████████████████████████████████████████████████████████████▉| 4999/5000 [11:12:00<00:03,  3.45s/it]


55.33780918727905

276.6783505154641


100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [11:12:12<00:00,  8.07s/it]


訓練次數5000，總回報344.5931677018625


In [ ]:
# Agent.Record()